In [1]:
import pyspark 
import argparse 
import json 

# 4.1.1 E: Top n frequent words in the review text

sc_conf = pyspark.SparkConf()
sc_conf.setAppName('task1')  
sc_conf.setMaster('local[*]') 
sc_conf.set('spark.driver.memory', '8g')
sc_conf.set('spark.executor.memory', '4g')
# sc = pyspark.SparkContext(conf=sc_conf)
sc = pyspark.SparkContext.getOrCreate()
sc.setLogLevel("OFF")

# read in the lines and create the rdd 
lines = sc.textFile("./data/review.json")

# loading the json into a python object 
rdd = lines.map(lambda line: json.loads(line))

your 131072x1 screen size is bogus. expect trouble


23/02/03 10:56:06 WARN Utils: Your hostname, LAPTOP-0S5927HR resolves to a loopback address: 127.0.1.1; using 192.168.250.233 instead (on interface eth0)
23/02/03 10:56:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/03 10:56:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
punctuations = ["(", "[", ",", ".", "!", "?", ":", ";", "]", ")"]

In [3]:
stopwords = []
with open('./data/stopwords') as f: 
    stopwords.append(f.read().split('\n'))
print(stopwords[0:10]) 

[['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'n

In [5]:
text = rdd.map(lambda x: x['text'])

In [6]:
splitting_words = text.map(lambda x: x.lower())

In [7]:
splitting_words.take(1) 

["walked in around 4 on a friday afternoon, we sat at a table just off the bar and walked out after 5 min or so. don't even think they realized we walked in. however everyone at the bar noticed we walked in!!! service was non existent at best. not a good way for a new business to start out. oh well, the location they are at has been about 5 different things over the past several years, so they will just be added to the list. smdh!!!"]

In [30]:
# first we need to filter by character 
counts = splitting_words.map(lambda x: x.split(' '))

In [31]:
counts.take(1)

[['walked',
  'in',
  'around',
  '4',
  'on',
  'a',
  'friday',
  'afternoon,',
  'we',
  'sat',
  'at',
  'a',
  'table',
  'just',
  'off',
  'the',
  'bar',
  'and',
  'walked',
  'out',
  'after',
  '5',
  'min',
  'or',
  'so.',
  "don't",
  'even',
  'think',
  'they',
  'realized',
  'we',
  'walked',
  'in.',
  'however',
  'everyone',
  'at',
  'the',
  'bar',
  'noticed',
  'we',
  'walked',
  'in!!!',
  'service',
  'was',
  'non',
  'existent',
  'at',
  'best.',
  'not',
  'a',
  'good',
  'way',
  'for',
  'a',
  'new',
  'business',
  'to',
  'start',
  'out.',
  'oh',
  'well,',
  'the',
  'location',
  'they',
  'are',
  'at',
  'has',
  'been',
  'about',
  '5',
  'different',
  'things',
  'over',
  'the',
  'past',
  'several',
  'years,',
  'so',
  'they',
  'will',
  'just',
  'be',
  'added',
  'to',
  'the',
  'list.',
  'smdh!!!']]

In [8]:
counts = splitting_words.flatMap(lambda line: line.split(' ')) \
    .filter(lambda x: x not in stopwords) \
    .map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)

In [16]:
counts = splitting_words.flatMap(lambda line: line.split(' ')) \
    .filter(lambda x: x not in stopwords) 


In [18]:
print(counts.take(3))
# counts = counts.filter(lambda x: x not in stopwords) 
# print(counts.collect()[0:3])    

['walked', 'in', 'around']


In [11]:
# counts = counts.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)
# print(counts.collect()[0:3])    

In [12]:
final = counts.sortBy(lambda x: x[1], False)
print(counts.collect()[0:3])    

[('even', 215856), ('they', 994327), ('noticed', 18323)]


In [13]:
# print(final.collect()[0:3])